In [1]:
%load_ext autoreload
%autoreload 2
import spacy
import scispacy
from scispacy.linking import EntityLinker
import en_core_sci_lg
from scify.nlp import *
from scify.utils.GNBR_preprocessing import get_data_and_distributions

UMLS is really big. Better to load linkers in separate cells. Sometimes crashes or similar on my Macbook Air

In [ ]:
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.

#UMLS is big and takes a while EVEN when CACHED it is downloading from tfidf_vectors from here idk why:
#url: /ai2-s2-scispacy/data/umls/tfidf_vectors_sparse.npz
linker_umls = EntityLinker(resolve_abbreviations=True, name="umls")

#linker_go = EntityLinker(resolve_abbreviations=True, name="go")
#linker_mesh = EntityLinker(resolve_abbreviations=True, name="mesh")

/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [ ]:
#linkers = [EntityLinker(resolve_abbreviations=True, name=ontology) for ontology in ["umls", "go", "mesh"]] this might crash
#linkers = [linker_mesh, linker_go, linker_umls]
#[get_ontology_name(linker) for linker in linkers]

#NER_models = [en_ner_bc5cdr_md, en_ner_craft_md, en_ner_bionlp13cg_md, en_ner_jnlpba_md]
#nlps = [model.load() for model in NER_models]

In [ ]:
#Specialized NER models trained on specialized corpus
import en_ner_bc5cdr_md
import en_ner_craft_md
import en_ner_bionlp13cg_md
import en_ner_jnlpba_md

In [ ]:
text="Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily."

I'm loading models here in every cell (instead of mapping them) separately because it sometimes crashes...

In [ ]:
base_nlp = spacy.load("en_core_sci_lg")

In [ ]:
nlp1 = spacy.load("en_ner_bc5cdr_md")

In [ ]:
nlp2 = spacy.load("en_ner_craft_md") #craft has loading issues? buggy....

In [ ]:
nlp3 = en_ner_bionlp13cg_md.load()

In [ ]:
nlp4 = en_ner_jnlpba_md.load()

In [ ]:
NER_nlps = [nlp1, nlp2, nlp3, nlp4]

In [ ]:
nlps = add_pipes_mutative(NER_nlps, linker_umls)

In [ ]:
base_nlp.pipeline, nlps[3].pipeline, nlps[0].meta["name"]

Be Very careful to overwrite Ents .... there's many attributes that are needed! I just attach an extension
(but in the end, my algorithm just looks at labels_ to determine if a sentence is worthy of pattern matching!)

1) _token the product(NER modules x mesh/go/umls)
2) Read the Text with all of them
3) Extract their labels and ids + confidence scores
4) Look at set differences -> overlap intersect?
4) Add labels to doc @ Scispacy_lg (no NER) so it has Labels (set_extension)
5) Assign highest confidence label to scispacy large

In [ ]:
#linker_umls.kb.cui_to_entity["5372017"]
#D016472 is a mesh ID

In [ ]:
chem_gene_in_texts = [
    "The antiarrhythmic effect of carvedilol was examined in a placebo-controlled multicenter trial , the Carvedilol Post-Infarct Survival Control in Left_Ventricular_Dysfunction -LRB- CAPRICORN -RRB- study , which enrolled 1,959 patients with reduced left ventricular systolic function after AMI , 98 % of whom were treated with an ACE inhibitor .",
   "Carvedilol improves survival of patients suffering from CHF but the effects of the drug on angiotensin-converting_enzyme -LRB- ACE -RRB- activity , renin and aldosterone are not well characterized in patients receiving an ACE inhibitor .",
"We therefore assessed the effects of introducing carvedilol and bisoprolol in a prospective manner on indices of oxidative stress -LSB- lipid hydroperoxides -LRB- LHP -RRB- -RSB- , endothelial damage -LSB- von_Willebrand factor -LRB- vWf -RRB- -RSB- , platelet activation -LRB- soluble P-selectin -RRB- and coagulation -LRB- fibrinogen -RRB- and their inter-relationships in stable outpatients with CHF in sinus rhythm ."
]

In [ ]:
%time
docs = get_merged_docs_for_texts(chem_gene_in_texts, base_nlp, nlps)

In [ ]:
visualise_doc(docs[0], compact=True)

In [ ]:
from scify.nlp import construct_pattern, prep_pattern

In [ ]:
construct_pattern(prep_pattern("kinase|compound|START_ENTITY RAFTK|nsubj|kinase RAFTK|dep|is is|nsubj|kinase kinase|nmod|activation activation|nmod|END_ENTITY"))

In [ ]:
sent1 = """In this study , we report that the related adhesion focal tyrosine kinase RAFTK , is an upstream kinase in beta1 integrin mediated activation of Akt."""

In [ ]:
visualise_doc(base_nlp(sent1))

In [ ]:
#get_non_zero support for one of the 30 themes only
#dependency paths + support
#Maybe merge dependency paths
#distributions

#it's very big but looks like this
distributions_kv_example = {"kinases|compound|START_ENTITY participate|nsubj|kinases participate|nmod|END_ENTITY": {'A+': 2.0,
  'A+.ind': 0.0,
  'A-': 0.0,
  'A-.ind': 0.0,
  'B': 0.0,
  'B.ind': 0.0,
  'E+': 0.0,
  'E+.ind': 0.0,
  'E-': 0.0,
  'E-.ind': 0.0,
  'E': 0.0,
  'E.ind': 0.0,
  'N': 0.0,
  'N.ind': 0.0,
  'O': 2.0,
  'O.ind': 0.0,
  'K': 0.0,
  'K.ind': 0.0,
  'Z': 0.0,
  'Z.ind': 0.0}}

In [ ]:
from scify.nlp import rgetattr, rsetattr
linker_umls.kb.cui_to_entity["C1413931"]

In [ ]:
[annotation["label"] 
 for ent in docs[0].ents 
 for annotation in ent._.annotated]

In [ ]:
[doc for doc in docs if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

In [ ]:
import spacy #from spacy.tokens import DocBin
import pickle
doc_data = pickle.dumps(docs) #only pickle lists of docs so it only includes their shared vocabulary
with open('../data/pubmed_annotated.pickle', 'wb') as f:
    pickle.dump(doc_data, f)
print(len(doc_data)) #that's a lot for 3 sentences

In [ ]:
doc_data2 = pickle.dumps(docs[1:]) #only pickle lists of docs so it only includes their shared vocabulary
with open('../data/pubmed_annotated.pickle', 'wb') as f:
    pickle.dump(doc_data, f)
print(len(doc_data2)) #ok but it's only the vocab and scales well!

In [ ]:
data, distributions = get_data_and_distributions()

In [ ]:
sents = [dat["sent"] for dat in data]


In [ ]:
len(sents)
sent_sample = sents[:500]

In [ ]:
docs = get_merged_docs_for_texts(sent_sample, base_nlp, nlps)

In [ ]:
wanted_sents = [doc for doc in docs if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

Let's see how well our NER modules work. Theoretically every sentence in the data should have at least one chemical and one gene entity in it

Ok that's 40% coverage....that's kinda bad

In [ ]:
len(wanted_sents), wanted_sents[200]


Let's check for patterns


In [ ]:
patterns = [d["dep"] for d in data[:500]]

constructed = [construct_pattern(pattern) for pattern in patterns]

len([con for con in constructed if con != None])

I can live with that ratio 430/500 (maybe errors not evenly distributed)

In [ ]:
import os
os.listdir("../data/pubmed")

In [ ]:
from pubmed_parser import parse_medline_xml
pubmed_abstracts = parse_medline_xml("../data/pubmed/pubmed20n1015.xml")
abstr = [article["abstract"] for article in pubmed_abstracts]

In [ ]:
%time
#already in pickle
#docs_pubmed = get_merged_docs_for_texts(abstr, base_nlp, nlps)

In [ ]:
#import spacy #from spacy.tokens import DocBin
import pickle

def w():
    doc_data = pickle.dumps(docs_pubmed) #only pickle lists of docs so it only includes their shared vocabulary
    with open('../data/pubmed_annotated.pickle', 'wb') as f:
        pickle.dump(doc_data, f)
    print(len(doc_data)) #that's a lot for 3 sentencesv
    


In [ ]:
[sent.ents for sent in dd.sents]
from typeguard import typechecked

from scify.nlp import *

[doc_has_entity_labels(sent, [["EE", "EE"], ["rr", "rr"]]) for sent in dd.sents]

In [ ]:
wanted_abstr = [doc for doc in docs_pubmed if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

In [ ]:

len([sent for abstract in wanted_abstr for sent in abstract.sents ]), len([sent for abstract in wanted_abstr for sent in abstract.sents if doc_has_entity_labels(sent,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])])

In [ ]:
pubmed_sents =[sent for abstract in wanted_abstr for sent in abstract.sents if doc_has_entity_labels(sent,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]
pubmed_sents[22]

In [ ]:
visualise_doc(pubmed_sents[22])

In [ ]:
show_noun_chunks(pubmed_sents[22])

In [ ]:
def get_shortest_dep_path(ent_a, ent_b):
    #networkx

def pattern_from_shortest_dep_path():
    5
    
def extract_pattern_between_entities(doc, ent_a, ent_b):
    4
    


In [ ]:
from scify.networks import get_edges
import networkx as nx
get_edges(pubmed_sents[22])


In [ ]:
doc = docs_pubmed[22]



In [ ]:
d

In [ ]:
from itertools import product

[* product(["Yolo", "Fredo", "checko"], ["fatty", "fagg"])]
tokens_lookup = {tok.text:tok for tok in doc}

In [ ]:
#TODO
#token -> char overlap in merge_docs
#document level matching %
#sentence level matching %
#performance?

#get_shortest_path between ents
#find pattern matches
#dependency pattern + pattern matcher on GNBR --> relation extraction precision?


In [ ]:
for token in doc[:25]:
    print(token, token.i, token.ent_iob)
print([noun for noun in doc[:25].noun_chunks])

In [ ]:
from spacy.tokens import Span
def semantic_spans_from_doc(doc):
    """returns a list of all spans eg. [token, token, entitiy (spanning more tokens), token]
    This is so you can iterate through it under one span API
    
    iobs
    -3 begins
    -2 outside
    -1 inside
    -0 no tag of
    ...entity
    """
    "3 1 1"
    "3 2 3 1"
    "2 3 1"
    spans = []
    current_span = []
    for idx, token in enumerate(doc):
        iob = token.ent_iob
        if (iob != 1):
            spans.append(Span)
            current_span.append()
        
        
    return spans, doc

In [ ]:
a = [1 ,2 ,3]

a.pop(0), a

In [ ]:
import copy
copy.deepcopy(doc)

In [ ]:
ent_idxs = [ent.start for ent in doc.ents]
non_ent_idxs = [token.i for token in doc if (token.ent_iob == 2 or token.ent_iob == 0)]


In [ ]:
sort(ent_idxs + non_ent_idx)


In [ ]:

spans =[]
ent_idxs = [ent.start for ent in doc.ents]
ents = doc.ents.copy()
for idx, token in enumerate(doc):
    if idx in ents_idxs:
        spans.append(doc[idx])
        
    


In [ ]:
[ent.start for ent in doc.ents],[token.i for token in doc if (token.ent_iob == 2 or token.ent_iob == 0)]


In [ ]:
[token.i for token in doc[:25] if token.ent_iob == 3],[token.i for token in doc[:25] if token.ent_iob != 2]

In [ ]:
#uniform 
#lowest_common_ancestor span

In [ ]:
#valid pattern
construct_pattern('|appos|START_ENTITY sensitivity|nmod|END_ENTITY') or []

In [ ]:
#invalid pattern
construct_pattern('sensitivity|nmod|END_ENTITY sensitivity|appos|START_ENTITY')

In [ ]:
from scify.networks import get_edges

TODO
1) isEntity in Sentence?
2) Shortest Path -> SP
3) compare with theme in GNBR